In [ ]:
import pandas as pd

df = pd.read_excel('/content/서울시 도시데이터 센서(S-DoT) 유동인구 설치 위치정보_251113.xlsx')
display(df.head())

,순번,방문자 센서코드,시리얼번호,주소,위도,경도
0,1,2992,V02Q1940942,서울특별시 양천구 신월동 805,37.532463,126.833076
1,2,2993,V02Q1940889,서울특별시 도봉구 창동 585-13,37.639946,127.036011
2,3,2994,V02Q1940947,서울특별시 동작구 상도동 518-7,37.499026,126.952012
3,4,2995,V02Q1940888,서울특별시 광진구 중곡동 265-5,37.563074,127.081995
4,5,2996,V02Q1940851,서울특별시 강동구 암사동 501-4,37.550391,127.128292


In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import warnings

warnings.filterwarnings('ignore')

start_date = date(2021, 12, 27)
target_date = date(2025, 7, 14)
curr = start_date

group1_dfs = []
group2_dfs = []
group3_dfs = []
group4_dfs = []

print("파일 로딩 및 데이터 정제(癤 제거) 시작...\n")

while curr <= target_date:
    base = f"S-DoT_WALK_{curr.strftime('%Y.%m.%d')}-{ (curr + timedelta(days=6)).strftime('%m.%d') }"
    path_csv = f"{base}.csv"
    path_xlsx = f"{base}.xlsx"

    df = None
    file_name = ""

    if date(2022, 1, 3) <= curr <= date(2023, 1, 1):
        group_idx = 1
        read_cols = range(8)
    elif date(2023, 1, 2) <= curr <= date(2023, 12, 24):
        group_idx = 2
        read_cols = None
    elif date(2023, 12, 25) <= curr <= date(2024, 1, 7):
        group_idx = 3
        read_cols = None
    elif date(2024, 1, 8) <= curr <= date(2025, 7, 14):
        group_idx = 4
        read_cols = None
    else:
        curr += timedelta(weeks=1)
        continue

    if os.path.exists(path_csv):
        file_name = path_csv
        try:
            df = pd.read_csv(path_csv, encoding='cp949', usecols=read_cols)
        except:
            try:
                df = pd.read_csv(path_csv, encoding='utf-8', usecols=read_cols)
            except Exception as e:
                print(f"FAIL (CSV): {path_csv} / {e}")

    elif os.path.exists(path_xlsx):
        file_name = path_xlsx
        try:
            df = pd.read_excel(path_xlsx, usecols=read_cols)
        except Exception as e:
            print(f"FAIL (XLSX): {path_xlsx} / {e}")

    if df is not None:
        df['source_file'] = file_name

        df = df.replace(to_replace=r'.*癤.*', value=np.nan, regex=True)

        if group_idx == 1:
            group1_dfs.append(df)
        elif group_idx == 2:
            group2_dfs.append(df)
        elif group_idx == 3:
            group3_dfs.append(df)
        elif group_idx == 4:
            group4_dfs.append(df)

    curr += timedelta(weeks=1)

def safe_concat(df_list, name):
    if df_list:
        res = pd.concat(df_list, ignore_index=True, sort=False)
        print(f"[{name}] 생성 완료 | 행: {len(res)}, 컬럼: {len(res.columns)}")
        return res
    else:
        print(f"[{name}] 파일 없음")
        return pd.DataFrame()

print("\n" + "-" * 30)
df_range1 = safe_concat(group1_dfs, "1구간 (22.01.03~)")
df_range2 = safe_concat(group2_dfs, "2구간 (23.01.02~)")
df_range3 = safe_concat(group3_dfs, "3구간 (23.12.25~)")
df_range4 = safe_concat(group4_dfs, "4구간 (24.01.08~)")

파일 로딩 및 데이터 정제(癤 제거) 시작...


------------------------------
[1구간 (22.01.03~)] 생성 완료 | 행: 3696053, 컬럼: 9
[2구간 (23.01.02~)] 생성 완료 | 행: 2945604, 컬럼: 9
[3구간 (23.12.25~)] 생성 완료 | 행: 104948, 컬럼: 7
[4구간 (24.01.08~)] 생성 완료 | 행: 7353755, 컬럼: 19


In [ ]:
if not df_range1.empty:
    print(f"[전처리 전] df_range1 행 개수: {len(df_range1)}")

    df_range1['날짜'] = df_range1['날짜'].astype(str).str[:8]

    df_range1['방문자수'] = pd.to_numeric(df_range1['방문자수'], errors='coerce').fillna(0)

    target_cols = ['기관 명', '모델명', '시리얼', '서버타입', '사이트명', '날짜']

    group_cols = [col for col in target_cols if col in df_range1.columns]

    df_range1_daily = df_range1.groupby(group_cols, as_index=False)['방문자수'].sum()

    df_range1 = df_range1_daily

    print(f"[전처리 후] df_range1 행 개수: {len(df_range1)} (일별 통합 완료)")
    print("\n[결과 미리보기]")
    print(df_range1.head())

[전처리 전] df_range1 행 개수: 3696053
[전처리 후] df_range1 행 개수: 29440 (일별 통합 완료)

[결과 미리보기]
  기관 명      모델명   시리얼        서버타입  사이트명        날짜   방문자수
0  서울시  SDOT001  2992  00Original  2992  20220102    434
1  서울시  SDOT001  2992  00Original  2992  20220103  22634
2  서울시  SDOT001  2992  00Original  2992  20220104  25393
3  서울시  SDOT001  2992  00Original  2992  20220105  25061
4  서울시  SDOT001  2992  00Original  2992  20220106  23883


In [ ]:
df_range1

,기관 명,모델명,시리얼,서버타입,사이트명,날짜,방문자수
0,서울시,SDOT001,2992,00Original,2992,20220102,434
1,서울시,SDOT001,2992,00Original,2992,20220103,22634
2,서울시,SDOT001,2992,00Original,2992,20220104,25393
3,서울시,SDOT001,2992,00Original,2992,20220105,25061
4,서울시,SDOT001,2992,00Original,2992,20220106,23883
...,...,...,...,...,...,...,...
29435,서울시,SDOT001,4063,00Original,4063,20221227,93
29436,서울시,SDOT001,4063,00Original,4063,20221228,88
29437,서울시,SDOT001,4065,00Original,4065,20221226,222
29438,서울시,SDOT001,4065,00Original,4065,20221227,414


In [ ]:
print(df_range4[df_range4['측정시간'].isnull()])

Empty DataFrame
Columns: [시리얼, 자치구, 행정동, 방문자수, 측정시간]
Index: []


In [ ]:
if not df_range2.empty:
    print(f"[전처리 전] df_range2 행 개수: {len(df_range2)}")

    df_range2['측정시간'] = df_range2['측정시간'].astype(str).str[:10]

    df_range2['방문자수'] = pd.to_numeric(df_range2['방문자수'], errors='coerce').fillna(0)

    potential_keys = ['기관 명', '지역', '시리얼', '자치구', '행정동', '측정시간']
    group_keys = [col for col in potential_keys if col in df_range2.columns]

    df_range2_daily = df_range2.groupby(group_keys, as_index=False)['방문자수'].sum()

    df_range2 = df_range2_daily

    print(f"[전처리 후] df_range2 행 개수: {len(df_range2)} (일별 통합 완료)")

[전처리 전] df_range2 행 개수: 2945604
[전처리 후] df_range2 행 개수: 25617 (일별 통합 완료)


In [ ]:
df_range2

,지역,시리얼,자치구,행정동,측정시간,방문자수
0,commercial_area,4051,Gangdong-gu,Buam-dong,2023-01-01,4
1,commercial_area,4051,Gangdong-gu,Buam-dong,2023-01-02,3971
2,commercial_area,4051,Gangdong-gu,Buam-dong,2023-01-03,4125
3,commercial_area,4051,Gangdong-gu,Buam-dong,2023-01-04,4475
4,commercial_area,4051,Gangdong-gu,Buam-dong,2023-01-05,3989
...,...,...,...,...,...,...
25612,traditional_markets,4049,Seodaemun-gu,Hongje3-dong,2023-12-20,2723
25613,traditional_markets,4049,Seodaemun-gu,Hongje3-dong,2023-12-21,2355
25614,traditional_markets,4049,Seodaemun-gu,Hongje3-dong,2023-12-22,3158
25615,traditional_markets,4049,Seodaemun-gu,Hongje3-dong,2023-12-23,3224


In [ ]:
if not df_range3.empty:
    print(f"[전처리 전] df_range3 행 개수: {len(df_range3)}")

    required_cols = ['자치구', '행정동', '방문자수', '측정시간']

    missing = [c for c in required_cols if c not in df_range3.columns]

    if not missing:
        df_range3['측정시간'] = df_range3['측정시간'].astype(str).str[:10]

        df_range3['방문자수'] = pd.to_numeric(df_range3['방문자수'], errors='coerce').fillna(0)

        df_range3_daily = df_range3.groupby(['자치구', '행정동', '측정시간'], as_index=False)['방문자수'].sum()

        df_range3 = df_range3_daily[required_cols]

        print(f"[전처리 후] df_range3 행 개수: {len(df_range3)} (일별 통합 완료)")
        print("\n[결과 미리보기]")
        print(df_range3.head())

[전처리 전] df_range3 행 개수: 104948
[전처리 후] df_range3 행 개수: 675 (일별 통합 완료)

[결과 미리보기]
         자치구          행정동  방문자수        측정시간
0  Dobong-gu  Chang1-dong   200  2023-12-25
1  Dobong-gu  Chang1-dong   234  2023-12-26
2  Dobong-gu  Chang1-dong   275  2023-12-27
3  Dobong-gu  Chang1-dong   242  2023-12-28
4  Dobong-gu  Chang1-dong   315  2023-12-29


In [ ]:
df_range3

,자치구,행정동,방문자수,측정시간
0,Dobong-gu,Chang1-dong,200,2023-12-25
1,Dobong-gu,Chang1-dong,234,2023-12-26
2,Dobong-gu,Chang1-dong,275,2023-12-27
3,Dobong-gu,Chang1-dong,242,2023-12-28
4,Dobong-gu,Chang1-dong,315,2023-12-29
...,...,...,...,...
670,Yongsan-gu,Itaewon2-dong,794,2024-01-03
671,Yongsan-gu,Itaewon2-dong,756,2024-01-04
672,Yongsan-gu,Itaewon2-dong,831,2024-01-05
673,Yongsan-gu,Itaewon2-dong,1151,2024-01-06


In [ ]:
if not df_range4.empty:
    print(f"[전처리 전] df_range4 행 개수: {len(df_range4)}")

    if '측정시간' in df_range4.columns:
        df_range4['측정시간'] = df_range4['측정시간'].astype(str).str[:10]
    elif '날짜' in df_range4.columns:
        df_range4['측정시간'] = df_range4['날짜'].astype(str).str[:10]

    df_range4['방문자수'] = pd.to_numeric(df_range4['방문자수'], errors='coerce').fillna(0)

    group_keys = ['시리얼', '자치구', '행정동', '측정시간']

    valid_keys = [k for k in group_keys if k in df_range4.columns]

    df_range4_daily = df_range4.groupby(valid_keys, as_index=False)['방문자수'].sum()

    final_cols = ['시리얼', '자치구', '행정동', '방문자수', '측정시간']
    available_cols = [c for c in final_cols if c in df_range4_daily.columns]

    df_range4 = df_range4_daily[available_cols]

    print(f"[전처리 후] df_range4 행 개수: {len(df_range4)} (일별 통합 완료)")
    print("\n[결과 미리보기]")
    print(df_range4.head())

[전처리 전] df_range4 행 개수: 7353755
[전처리 후] df_range4 행 개수: 51708 (일별 통합 완료)

[결과 미리보기]
    시리얼           자치구               행정동   방문자수        측정시간
0  2992  Yangcheon-gu  Sinwol1(il)-dong  14106  2024-08-20
1  2992  Yangcheon-gu  Sinwol1(il)-dong  34783  2024-08-21
2  2992  Yangcheon-gu  Sinwol1(il)-dong  34099  2024-08-22
3  2992  Yangcheon-gu  Sinwol1(il)-dong  36634  2024-08-23
4  2992  Yangcheon-gu  Sinwol1(il)-dong  34166  2024-08-24


In [ ]:
df_range4

,시리얼,자치구,행정동,방문자수,측정시간
0,2992,Yangcheon-gu,Sinwol1(il)-dong,14106,2024-08-20
1,2992,Yangcheon-gu,Sinwol1(il)-dong,34783,2024-08-21
2,2992,Yangcheon-gu,Sinwol1(il)-dong,34099,2024-08-22
3,2992,Yangcheon-gu,Sinwol1(il)-dong,36634,2024-08-23
4,2992,Yangcheon-gu,Sinwol1(il)-dong,34166,2024-08-24
...,...,...,...,...,...
51703,4065,Seoul_Grand_Park,trail,351,2025-07-16
51704,4065,Seoul_Grand_Park,trail,131,2025-07-17
51705,4065,Seoul_Grand_Park,trail,244,2025-07-18
51706,4065,Seoul_Grand_Park,trail,251,2025-07-19


In [ ]:
if not df_range4.empty and not df_range3.empty:
    print("[매핑 작업] df_range4의 정보를 기반으로 df_range3에 '시리얼' 추가 중...")

    mapping_source = df_range4[['자치구', '행정동', '시리얼']].copy()

    mapping_table = mapping_source.drop_duplicates(subset=['자치구', '행정동'], keep='last')

    print(f"  - 매핑 기준 데이터 수: {len(mapping_table)}개 지역")

    df_range3_mapped = pd.merge(df_range3, mapping_table, on=['자치구', '행정동'], how='left')

    df_range3 = df_range3_mapped

    cols = ['시리얼', '자치구', '행정동', '방문자수', '측정시간']

    final_cols = [c for c in cols if c in df_range3.columns]
    df_range3 = df_range3[final_cols]

    print("[완료] df_range3 시리얼 매핑 완료")
    print(df_range3.head())

    missing_count = df_range3['시리얼'].isnull().sum()
    if missing_count > 0:
        print(f"[주의] df_range4에 정보가 없어 매핑되지 않은 행 개수: {missing_count}")

[매핑 작업] df_range4의 정보를 기반으로 df_range3에 '시리얼' 추가 중...
  - 매핑 기준 데이터 수: 130개 지역
[완료] df_range3 시리얼 매핑 완료
    시리얼        자치구          행정동  방문자수        측정시간
0  4028  Dobong-gu  Chang1-dong   200  2023-12-25
1  4028  Dobong-gu  Chang1-dong   234  2023-12-26
2  4028  Dobong-gu  Chang1-dong   275  2023-12-27
3  4028  Dobong-gu  Chang1-dong   242  2023-12-28
4  4028  Dobong-gu  Chang1-dong   315  2023-12-29


In [ ]:
import pandas as pd

final_columns = ['시리얼', '자치구', '행정동', '측정시간', '방문자수']

print("통합 작업 시작...")

if not df_range4.empty:
    location_map = df_range4[['시리얼', '자치구', '행정동']].drop_duplicates(subset=['시리얼'], keep='last')
    print(f"  - 위치 매핑 정보 생성 완료 (총 {len(location_map)}개 기기 정보)")
else:
    location_map = pd.DataFrame(columns=['시리얼', '자치구', '행정동'])
    print("  - [주의] df_range4가 비어있어 위치 정보를 매핑할 수 없습니다.")

if not df_range1.empty:
    df1 = df_range1.rename(columns={'날짜': '측정시간'})
    df1 = pd.merge(df1, location_map, on='시리얼', how='left')
    df1 = df1.reindex(columns=final_columns)
    print(f"  - 1구간 준비 완료 ({len(df1)}행)")
else:
    df1 = pd.DataFrame(columns=final_columns)

if not df_range2.empty:
    df2 = df_range2.copy()
    df2 = pd.merge(df2, location_map, on='시리얼', how='left')
    df2 = df2.reindex(columns=final_columns)
    print(f"  - 2구간 준비 완료 ({len(df2)}행)")
else:
    df2 = pd.DataFrame(columns=final_columns)

if not df_range3.empty:
    df3 = df_range3.reindex(columns=final_columns)
    print(f"  - 3구간 준비 완료 ({len(df3)}행)")
else:
    df3 = pd.DataFrame(columns=final_columns)

if not df_range4.empty:
    df4 = df_range4.reindex(columns=final_columns)
    print(f"  - 4구간 준비 완료 ({len(df4)}행)")
else:
    df4 = pd.DataFrame(columns=final_columns)


master_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

if not master_df.empty:
    master_df = master_df.sort_values(by='측정시간')

print("-" * 40)
print(f"통합 완료: master_df 생성됨")
print(f"총 행 개수: {len(master_df)}")
print(f"컬럼 목록: {list(master_df.columns)}")
print("-" * 40)

missing_loc = master_df['자치구'].isnull().sum()
if missing_loc > 0:
    print(f"[참고] 위치 정보(자치구/행정동)가 없는 데이터: {missing_loc}건")
    print("       (df_range4에 해당 시리얼 번호가 없어 매핑되지 않은 과거 데이터일 수 있습니다.)")

file_name = "S-DoT_WALK_Master_Integrated.csv"
try:
    master_df.to_csv(file_name, index=False, encoding='utf-8-sig')
    print(f"\n[성공] 파일 저장 완료: {file_name}")

    print(master_df.head(10))
except Exception as e:
    print(f"\n[실패] 파일 저장 중 오류 발생: {e}")

통합 작업 시작...
  - 위치 매핑 정보 생성 완료 (총 106개 기기 정보)
  - 1구간 준비 완료 (29440행)
  - 2구간 준비 완료 (25617행)
  - 3구간 준비 완료 (675행)
  - 4구간 준비 완료 (51708행)
----------------------------------------
통합 완료: master_df 생성됨
총 행 개수: 107440
컬럼 목록: ['시리얼', '자치구', '행정동', '측정시간', '방문자수']
----------------------------------------
[참고] 위치 정보(자치구/행정동)가 없는 데이터: 25933건
       (df_range4에 해당 시리얼 번호가 없어 매핑되지 않은 과거 데이터일 수 있습니다.)

[성공] 파일 저장 완료: S-DoT_WALK_Master_Integrated.csv
        시리얼  자치구  행정동      측정시간  방문자수
16596  4012  NaN  NaN  20210824    10
26775  4044  NaN  NaN  20210910     0
26776  4044  NaN  NaN  20210911     0
26777  4044  NaN  NaN  20210912     0
26778  4044  NaN  NaN  20210913     0
26779  4044  NaN  NaN  20210914     0
26780  4044  NaN  NaN  20210915     0
26781  4044  NaN  NaN  20210916     0
26782  4044  NaN  NaN  20210917     0
26783  4044  NaN  NaN  20210918     0


In [ ]:
master_df

,시리얼,자치구,행정동,측정시간,방문자수
16596,4012,NaN,NaN,20210824,10
26775,4044,NaN,NaN,20210910,0
26776,4044,NaN,NaN,20210911,0
26777,4044,NaN,NaN,20210912,0
26778,4044,NaN,NaN,20210913,0
...,...,...,...,...,...
99914,4045,Seoul_Grand_Park,valet_parking1,20250720,6122
65121,3011,Gangnam-gu,Gaepo2(i)-dong,20250720,38257
64597,3010,Dobong-gu,Chang2(i)-dong,20250720,28778
74841,3030,Geumcheon-gu,Gasan-dong,20250720,30231


In [ ]:
print(master_df[master_df['측정시간'].isnull()])

Empty DataFrame
Columns: [시리얼, 자치구, 행정동, 측정시간, 방문자수]
Index: []


In [ ]:
import pandas as pd

master_df['측정시간'] = master_df['측정시간'].astype(str).str.replace(r'\D', '', regex=True)

master_df['측정시간'] = pd.to_numeric(master_df['측정시간'], errors='coerce')

master_df['측정시간'] = master_df['측정시간'].astype('Int64')

print("--- 변환 결과 확인 ---")
print(master_df['측정시간'].head())

print("\n--- 데이터 샘플 및 타입 ---")
print(master_df.info())

--- 변환 결과 확인 ---
16596    20210824
26775    20210910
26776    20210911
26777    20210912
26778    20210913
Name: 측정시간, dtype: Int64

--- 데이터 샘플 및 타입 ---
<class 'pandas.core.frame.DataFrame'>
Index: 107440 entries, 16596 to 107439
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   시리얼     107440 non-null  int64 
 1   자치구     81507 non-null   object
 2   행정동     81507 non-null   object
 3   측정시간    107440 non-null  Int64 
 4   방문자수    107440 non-null  int64 
dtypes: Int64(1), int64(2), object(2)
memory usage: 5.0+ MB
None


In [ ]:
import pandas as pd

master_df['측정시간'] = master_df['측정시간'].astype(str).str.replace(r'\D', '', regex=True)
master_df['측정시간'] = pd.to_numeric(master_df['측정시간'], errors='coerce').astype('Int64')

mapping_cols = ['방문자 센서코드', '주소', '위도', '경도']

final_df = pd.merge(
    master_df,
    df[mapping_cols],
    left_on='시리얼',
    right_on='방문자 센서코드',
    how='left'
)

target_columns = ['시리얼', '주소', '위도', '경도', '측정시간', '방문자수']
final_df = final_df[target_columns]



print("--- final_df 생성 완료 ---")
print(final_df.info())
print("\n--- 데이터 샘플 (상위 5개) ---")
print(final_df.head())

if final_df['주소'].isna().sum() > 0:
    print(f"\n[참고] '시리얼'과 '방문자 센서코드'가 매칭되지 않아 주소가 없는 데이터가 {final_df['주소'].isna().sum()}건 있습니다.")

--- final_df 생성 완료 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107440 entries, 0 to 107439
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   시리얼     107440 non-null  int64  
 1   주소      107125 non-null  object 
 2   위도      107125 non-null  float64
 3   경도      107125 non-null  float64
 4   측정시간    107440 non-null  Int64  
 5   방문자수    107440 non-null  int64  
dtypes: Int64(1), float64(2), int64(2), object(1)
memory usage: 5.0+ MB
None

--- 데이터 샘플 (상위 5개) ---
    시리얼                  주소         위도         경도      측정시간  방문자수
0  4012  서울특별시 종로구 세종로 1-68  37.574628  126.97651  20210824    10
1  4044                 NaN        NaN        NaN  20210910     0
2  4044                 NaN        NaN        NaN  20210911     0
3  4044                 NaN        NaN        NaN  20210912     0
4  4044                 NaN        NaN        NaN  20210913     0

[참고] '시리얼'과 '방문자 센서코드'가 매칭되지 않아 주소가 없는 데이터가 315건 있습니다.


In [ ]:
final_df

,시리얼,주소,위도,경도,측정시간,방문자수
0,4012,서울특별시 종로구 세종로 1-68,37.574628,126.976510,20210824,10
1,4044,NaN,NaN,NaN,20210910,0
2,4044,NaN,NaN,NaN,20210911,0
3,4044,NaN,NaN,NaN,20210912,0
4,4044,NaN,NaN,NaN,20210913,0
...,...,...,...,...,...,...
107435,4045,서울대공원 발레주차장-1,37.433454,127.009297,20250720,6122
107436,3011,서울특별시 강남구 개포동 186-19,37.489030,127.067883,20250720,38257
107437,3010,서울특별시 도봉구 창동 445-22,37.640683,127.035490,20250720,28778
107438,3030,서울특별시 금천구 가산동 가산로5길 43,37.473511,126.892135,20250720,30231


In [ ]:
final_df.to_csv('유동인구데이터', index=False, encoding='utf-8-sig')